# Linear diffusion revisited

We are interested in solving the problem when there is periodic forcing $j=\hat{j}\mathrm{e}^{i\omega t}$. We look for a solution $c=\hat{c}\mathrm{e}^{i\omega t}$ so that our problem becomes 

$$i \omega \hat{c} = D \hat{c}_{xx}, \quad \left. \hat{c}_x \right|_{x=0} = 0, \quad \left. -D\hat{c}_x \right|_{x=1}=\hat{j}.$$

After discretisation using the finite volume method the problem will look like

$$ i \omega \boldsymbol{\hat{c}} = J \boldsymbol{\hat{c}} + \boldsymbol{b},$$

where

$$ 
J = \frac{D}{(\Delta x)^2} \begin{bmatrix}
-1 & 1 &  0 &  & & \\
1 & -2 & 1 &  & & \\
 & 1 & -2 & 1 & & \\
 &  & \ddots & \ddots & \ddots& \\
 &  & & 1 & -2 & 1  \\
 & &  & 0 & 1 & -1 
\end{bmatrix}
$$
and
$$
\boldsymbol{b} = \begin{bmatrix}
0 \\
\vdots \\
0 \\
-\hat{j}/\Delta x
\end{bmatrix}
$$

Here we will show how to extract these matrices from the model

## Building the model
As before, we start by importing PyBaMM, along with any other packages we require.

In [1]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt

We then load up an instance of the `pybamm.BaseModel` class.

In [2]:
model = pybamm.BaseModel()

We now define the model variables and parameters. We also define the spatial variable $x$ here. Since we are solving in 1D we have decided to call the domain "rod", but we could name it anything we like. Note that in PyBaMM variables and parameters can be given useful and meaningful names, such as "Concentration", so that they can be easily referred to later. Values for the parameters will be provided later.

In [3]:
x = pybamm.SpatialVariable("x", domain="rod", coord_sys="cartesian")
c_hat = pybamm.Variable("Concentration (FT)", domain="rod")
D = pybamm.Parameter("Diffusivity")
c0 = pybamm.Parameter("Initial concentration")

Now the time dependence of $j$ is fixed, so $\hat{j}$ is just a parameter

In [4]:
j_hat = pybamm.Parameter("Applied flux (FT)")

Now that we have defined the variables, we can write down the model equations and add them to the `model.rhs` dictionary. This dictionary stores the right hand sides of any time-dependent differential equations (ordinary or partial). The key is the variable inside the time derivative (in this case $c$).

In [5]:
N = - D * pybamm.grad(c_hat)  # The flux 
dcdt = - pybamm.div(N)  # The right hand side of the PDE
model.rhs = {c_hat: dcdt}  # Add to model

We now add the boundary conditions into the `model.boundary_conditions` dictionary. The keys of the dictionary indicate which end of the boundary the condition is applied to (in 1D this can be "left" or "right"), the entry is then give as a tuple of the value and type.

In [6]:
model.boundary_conditions = {
    c_hat: {
        "left": (pybamm.Scalar(0), "Neumann"),
        "right": (-j_hat / D, "Neumann"),
    }
}

We also need to add the initial conditions to the `model.initial_conditions` dictionary (we don't need this now, but PyBaMM will throw an error if we don't provide an initial condition).

In [7]:
model.initial_conditions = {c_hat: c0}

Finally, we add any output variables to the `model.variables` dictionary. These variables can be easily accessed after the model has been solved. You can add any variables of interest to this dictionary.

In [8]:
model.variables = {
    "Concentration (FT)": c_hat, 
    "Applied flux (FT)": j_hat,
}

## Using the model
Now that the model has been constructed we can go ahead and define our geometry and parameter values. We start by defining the geometry for our "rod" domain. We need to define the spatial direction(s) in which spatial operators act (such as gradients). In this case it is simply $x$. We then set the minimum and maximum values $x$ can take. In this example we are solving the problem on the domain $0<x<1$.

In [9]:
geometry = {"rod": {x: {"min": pybamm.Scalar(0), "max": pybamm.Scalar(1)}}}

We also need to provide the values of any parameters using the `pybamm.ParameterValues` class. This class accepts a dictionary of parameter names and values. Note that the name we provide is the string name of the parameters and not its symbol.

We will set the problem up with all parameters set to 1 since it is easy to see how we need to modify the matrix $J$ and vector $b$ when the parameters change

In [10]:
# Define parameter values object
param = pybamm.ParameterValues(
    {
        "Applied flux (FT)": 1,
        "Diffusivity": 1,
        "Initial concentration": 1,
    },
)

Now that we have defined the geometry and provided the parameters values, we can process the model.

In [11]:
param.process_model(model)
param.process_geometry(geometry)

Before we disctretise the spatial operators, we must choose and mesh and a spatial method. Here we choose to use a uniformly spaced 1D mesh with 30 points, and discretise the equations in space using the finite volume method. The information about the mesh is stored in a `pybamm.Mesh` object, wheres the spatial methods are stored in a dictionary which maps domain names to a spatial method. This allows the user to discretise different (sub)domains in a problem using different spatial methods. All of this information goes into a `pybamm.Discretisation` object, which accepts a mesh and a dictionary of spatial methods.

In [12]:
submesh_types = {"rod": pybamm.Uniform1DSubMesh}
var_pts = {x: 8}
mesh = pybamm.Mesh(geometry, submesh_types, var_pts)
spatial_methods = {"rod": pybamm.FiniteVolume()}
disc = pybamm.Discretisation(mesh, spatial_methods)

The model is then processed using the discretisation, turning the spaital operators into matrix-vector multiplications.

In [13]:
disc.process_model(model)

Now we are ready to extract the matrix-vector problem.

## Extracting the matrix-vector problem

The matrix J is just the jacobian. In PyBaMM the Jacobian is a function since it may depend on state and time in general. The Jacobian is calculated in the solver set up stage, which we can execute as follows

In [14]:
solver = pybamm.ScipySolver()
solver.set_up(model)

Next we will arbitrarily evaluate using the initial state and time.

In [15]:
y0 = model.concatenated_initial_conditions.entries  # vector of initial conditions
J = model.jac_rhs_algebraic_eval(0, y0, []).sparse()  #  call the Jacobian and return a (sparse) matrix

For this problem, we can get the vector $b$ by evaluating the rhs of the ODE (i.e. the $J\boldsymbol{\hat{c}} + \boldsymbol{b})$ using a $\boldsymbol{\hat{c}}$ that is constant in space. We'll use our initial condition

In [16]:
b = model.rhs_algebraic_eval(0, y0, []).sparse()

We can look at $J$ and $\boldsymbol{b}$ and see we get the expected result

In [17]:
J.todense()

matrix([[ -64.,   64.,    0.,    0.,    0.,    0.,    0.,    0.],
        [  64., -128.,   64.,    0.,    0.,    0.,    0.,    0.],
        [   0.,   64., -128.,   64.,    0.,    0.,    0.,    0.],
        [   0.,    0.,   64., -128.,   64.,    0.,    0.,    0.],
        [   0.,    0.,    0.,   64., -128.,   64.,    0.,    0.],
        [   0.,    0.,    0.,    0.,   64., -128.,   64.,    0.],
        [   0.,    0.,    0.,    0.,    0.,   64., -128.,   64.],
        [   0.,    0.,    0.,    0.,    0.,    0.,   64.,  -64.]])

In [18]:
b.todense()

matrix([[ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [-8.]])

We can also get the mass matrix as follows

In [19]:
M = model.mass_matrix.entries

and construct the following problem 

$$ A \boldsymbol{\hat{c}} = \boldsymbol{b},$$

with $A = i \omega M - J$. Here the mass matrix is just the identity.

In [20]:
omega = 1
A = 1j * omega * M - J